In [ ]:
import os
username = 'recspert'
repo = 'ITP-SeqRecSys-2024'

# remove local directory if it already exists
if os.path.isdir(repo):
    !rm -rf {repo}

!git clone https://github.com/{username}/{repo}.git

In [ ]:
!pip install --no-cache-dir --upgrade git+https://github.com/evfro/polara.git@develop#egg=polara

In [ ]:
import numpy as np
import pandas as pd

from polara import get_movielens_data
from polara.preprocessing.dataframes import reindex, leave_one_out

from dataprep import transform_indices
from evaluation import topn_recommendations

from scipy.sparse import csr_matrix, coo_matrix
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

Develop a recommender systems framework for conducting the necessary experiments, which includes:
- training recsys models
- generating recommendations
- evaluating recommendations quality
- performing model comparison

## Experiment protocol can be described in terms of 4 functions:

```python
# building/training a recommender model
model_params = build_func(model_config, trainset, trainset_description)

# predicting relevance scores for test user-item pairs
model_scores = score_func(model_params, testset, testset_description)

# generating top-n recommendations using predcted scores
model_recoms = recom_func(model_scores, topn)

# evaluating quality of recommendations
recs_quality = evaluate_func(model_recoms, holdout, holdout_description)
```


**Essentially, this is the main functionality provided by most of the recommender systems frameworks.**  
That's why we can say that we're building a simple recsys framework from scratch.

In [ ]:
data = get_movielens_data(include_time=True)

data_description = {
    'users':'userid',
    'items':'movieid',
    'feedback':'rating',
    'timestamp':'timestamp'
}

In [ ]:
data.head()

# Data split

![alt text](./plots/split.png "Logo Title Text 1")

In [ ]:
# define the timepoint corresponding to the 95% percentile
test_timepoint = data['timestamp'].quantile(
    q=0.95, interpolation='nearest'
)

# interaction after timepoint go to test
test_data_ = data.query('timestamp >= @test_timepoint')
# interaction before timepoint go to train,
# also hiding the interactions of test users
# this ensures the warm-start strategy
train_data_ = data.query(
    'userid not in @test_data_.userid.unique() and timestamp < @test_timepoint'
)


![alt text](./plots/truncate.png "Logo Title Text 1")

In [ ]:
# transform user and item ids for convenience, reindex test data
training, data_index = transform_indices(train_data_.copy(), 'userid', 'movieid')
test_data = reindex(test_data_, data_index['items'], filter_invalid=False)

# the items that were not in the training set have itemid -1
# let's drop the items with itemid -1 and all consequtive interactions
test_data = test_data.sort_values(by=[data_description['users'], data_description['timestamp']])
mask = test_data.groupby(data_description['users']).cummin()[data_description['items']] == -1
test_data_truncated = test_data[~mask]

# also get rid of users who have just 1 interaction
interaction_counts = test_data_truncated.groupby(data_description['users']).size()
users_to_keep = interaction_counts[interaction_counts >= 2].index
test_prepared = test_data_truncated[test_data_truncated[data_description['users']].isin(users_to_keep)]



In [ ]:
testset, holdout = leave_one_out(
    test_prepared, target='timestamp', sample_top=True, random_state=0
)

# Evaluating recommendations quality

We will use **HitRate** (HR) и **Mean Reciprocal Rank** (MRR).  We will also compute **Coverage** (Cov) to evaluate the diversity of recommendations

*Note*: In the case of a single holdout item per user the latter coincides with the Average Reciprocal HitRate (ARHR) and Mean Average Precision (MAP).

$$
\text{HR} = \frac{1}{\text{\# test users}} \sum_{\text{test users}}{hit}, \quad
$$

$$
hit = 
\begin{gather*}
\begin{cases}
  1 & \text{if holdout item in top-$n$ recommendations,}\\    
  0 & \text{otherwise.}
\end{cases}
\end{gather*}
$$

$$
\text{MRR} = \frac{1}{\text{\# test users}} \sum_{\text{test users}}{\frac{1}{\text{hit rank}}}
$$

$$
\text{Cov} = \frac{\text{\# unique recommendations}}{\text{\# items in catalogue}}
$$

In [ ]:
def topn_recommendations(scores, topn=10):
    recommendations = np.apply_along_axis(topidx, 1, scores, topn)
    return recommendations


def topidx(a, topn):
    parted = np.argpartition(a, -topn)[-topn:]
    return parted[np.argsort(-a[parted])]

In [ ]:
def downvote_seen_items(scores, data, data_description):
    assert isinstance(scores, np.ndarray), 'Scores must be a dense numpy array!'
    itemid = data_description['items']
    userid = data_description['users']
    # get indices of observed data, corresponding to scores array
    # we need to provide correct mapping of rows in scores array into
    # the corresponding user index (which is assumed to be sorted)
    row_idx, test_users = pd.factorize(data[userid], sort=True)
    assert len(test_users) == scores.shape[0]
    col_idx = data[itemid].values
    # downvote scores at the corresponding positions
    scores[row_idx, col_idx] = scores.min() - 1

In [ ]:
def model_evaluate(recommended_items, holdout, holdout_description, topn=10):
    itemid = holdout_description['items']
    holdout_items = holdout[itemid].values
    assert recommended_items.shape[0] == len(holdout_items)
    hits_mask = recommended_items[:, :topn] == holdout_items.reshape(-1, 1)
    # HR calculation
    hr = ...
    # MRR calculation
    n_test_users = recommended_items.shape[0]
    hit_rank = np.where(hits_mask)[1] + 1.0
    mrr = ...
    # coverage calculation
    n_items = holdout_description['n_items']
    cov = ...
    return {'hr':hr, 'mrr':mrr, 'cov':cov}

In [ ]:
def build_evaluate_model(Model, model_config, data_dict, data_description):
    '''
    Builds the model and calculates metric using the holdout set.
    Args:
        Model (model class): The model class to train
        model config (dict): A dictionary with model hyperparameters
        data_dict (dict): A dictionary containing data with the following keys:
            - 'train' (pd.DataFrame): The input dataframe containing the train user-item interactions.
            - 'test' (pd.DataFrame): The input dataframe containing the test user-item interactions.
            - 'holdout' (pd.DataFrame): The input dataframe containing the holdout to measure the quality of recommendations.
        data_description (dict): A dictionary containing the data description with the following keys:
            - 'n_users' (int): The total number of unique users in the data.
            - 'n_items' (int): The total number of unique items in the data.
            - 'users' (str): The name of the column in the dataframe containing the user ids.
            - 'items' (str): The name of the column in the dataframe containing the item ids.
            - 'feedback' (str): The name of the column in the dataframe containing the user-item interaction feedback.
            - 'timestamp' (str): The name of the column in the dataframe containing the timestamps of interactions.

    Returns:
        np.array: A numpy array of shape (n_test_users, n_items) containing the scores for each user.
    '''
    model = Model(model_config)
    model.build(data_dict['train'], data_description)
    preds = model.recommend(data_dict['test'], data_description)
    downvote_seen_items(preds, data_dict['test'], data_description)
    recs = topn_recommendations(preds)
    metrics = model_evaluate(recs, data_dict['holdout'], data_description)

    return metrics, model

In [ ]:
class Random:
    def __init__(self, model_config=None) -> None:
        # for reproducibility, not a hyperparameter
        self.seed = model_config['seed']
        self.rng = np.random.default_rng(seed=model_config['seed'])
    
    def build(self, data, data_description):
        self.n_items = data.nunique()[data_description['items']]
        
    def recommend(self, data, data_description):
        n_users = data.nunique()[data_description['users']]
        n_items = ...
        scores = ...
        return scores

In [ ]:
class Popular:
    def __init__(self, model_config=None) -> None:
        pass
    
    def build(self, data, data_description):
        item_popularity = data[data_description['items']].value_counts()
        n_items = item_popularity.index.max() + 1
        popularity_scores = ...
        popularity_scores[...] = item_popularity.values
        self.popularity_scores = ...
        
    def recommend(self, data, data_description):
        n_users = data.nunique()[data_description['users']]
        scores = ...
        return scores

In [ ]:
data_dict = {
    'train':training,
    'test':testset,
    'holdout':holdout
}

In [ ]:
data_description = {
    'users':'userid',
    'items':'movieid',
    'feedback':'rating',
    'timestamp':'timestamp',
}

In [ ]:
rand_metrics, rand_model = build_evaluate_model(Model=Random, model_config={'seed':2024}, data=data_dict, data_description=data_description)
pop_metrics, pop_model = build_evaluate_model(Model=Popular, model_config={}, data=data_dict, data_description=data_description)

In [ ]:
pop_metrics

In [ ]:
rand_metrics

# Association rules

In [ ]:
def generate_interactions_matrix(data, data_description, rebase_users=False):
    '''
    Converts a pandas dataframe with user-item interactions into a sparse matrix representation.
    Allows reindexing user ids, which help ensure data consistency at the scoring stage
    (assumes user ids are sorted in the scoring array).
    
    Args:
        data (pandas.DataFrame): The input dataframe containing the user-item interactions.
        data_description (dict): A dictionary containing the data description with the following keys:
            - 'n_users' (int): The total number of unique users in the data.
            - 'n_items' (int): The total number of unique items in the data.
            - 'users' (str): The name of the column in the dataframe containing the user ids.
            - 'items' (str): The name of the column in the dataframe containing the item ids.
            - 'feedback' (str): The name of the column in the dataframe containing the user-item interaction feedback.
        rebase_users (bool, optional): Whether to reindex the user ids to make contiguous index starting from 0. Defaults to False.

    Returns:
        scipy.sparse.csr_matrix: A sparse matrix of shape (n_users, n_items) containing the user-item interactions.
    '''
    n_users = data_description['n_users']
    n_items = data_description['n_items']
    # get indices of observed data
    user_idx = data[data_description['users']].values
    if rebase_users: # handle non-contiguous index of test users
        # This ensures that all user ids are contiguous and start from 0,
        # which helps ensure data consistency at the scoring stage.
        user_idx, user_index = pd.factorize(user_idx, sort=True)
        n_users = len(user_index)
    item_idx = data[data_description['items']].values
    feedback = np.ones_like(item_idx)
    # construct rating matrix
    return csr_matrix((feedback, (user_idx, item_idx)), shape=(n_users, n_items), dtype='f8')

$$
\text{score}_{AR}(u, i) = \text{PairCount}_{AR}(i_{|I_u|}, i)
$$

$$
\text{PairCount}_{AR}(i, j) = |U_i\cap U_j|
$$

$I_u$ - interaction history of user $u$, $U_i$ - set of users who interacted with item $i$, $i_{|I_u|}$ - last item of user $u$

In [ ]:
class AR:
    def __init__(self, model_config=None) -> None:
        pass
    
    def build(self, data, data_description):
        '''
        Builds association rules matrix.
        '''
        self.rules = ...
        
    def recommend(self, data, data_description):
        '''
        Generate scores for given data.
        '''
        # Drop duplicates, keeping the last interaction for each user
        
        scores = ...
        return scores

$$
\text{score}_{SR}(u, i) = \text{PairCount}_{SR}(i_{|I_u|} \rightarrow i)
$$

$$
\text{PairCount}_{SR}(j \rightarrow i) = \sum_{v\in U} \textbf{1}[j\rightarrow_v i]
$$

where $j\rightarrow_u i$ means that item $i$ follows item $j$ in the interaction history of user $u$. $U$ is the set of users

In [ ]:
class SR:
    def __init__(self, model_config=None) -> None:
        pass
    
    def build(self, data, data_description):
        'Builds sequential rules of size two'
        rules = {}
        
        # get chronological interaction history for each user
        histories = (
            data
            .sort_values(
                by=data_description['timestamp']
                )
            .groupby(data_description['users'])[data_description['items']]
            .apply(list)
            )
        
        self.rules = ...

    def recommend(self, data, data_description):
        '''
        Generate scores for given data.
        '''
        scores = ...
        return scores


In [ ]:
ar_metrics, ar_model = build_evaluate_model(Model=AR, model_config={}, data=data_dict, data_description=data_description)
sr_metrics, sr_model = build_evaluate_model(Model=SR, model_config={}, data=data_dict, data_description=data_description)

In [ ]:
results = {
    'Sequential Rules':sr_metrics,
    'Association Rules':ar_metrics,
    'popular':pop_metrics,
    'random':rand_metrics
}

pd.DataFrame.from_dict(results, orient='index')

# Successive evaluation

![alt text](./plots/split.png "Logo Title Text 1")

In [ ]:
# define the timepoint corresponding to the 95% percentile
test_timepoint = data['timestamp'].quantile(
    q=0.95, interpolation='nearest'
)

# interaction after timepoint go to test
test_data_ = data.query('timestamp >= @test_timepoint')
# interaction before timepoint go to train,
# also hiding the interactions of test users
# this ensures the warm-start strategy
train_data_ = data.query(
    'userid not in @test_data_.userid.unique() and timestamp < @test_timepoint'
)

In [ ]:
test_timepoint

In [ ]:
train_data_

In [ ]:
test_data_

In [ ]:
# transform user and item ids for convenience, reindex test data
training, data_index = transform_indices(train_data_.copy(), 'userid', 'movieid')

# reindex items in test set, if item was not in train, assign -1 as itemid
test_data = reindex(test_data_, data_index['items'], filter_invalid=False)

# Successive evaluation

![alt text](./plots/eval.png "Logo Title Text 1")

In [ ]:
# the items that were not in the training set have itemid -1
# let's drop the items with itemid -1 and all consequtive interactions
test_data = test_data.sort_values(by=[data_description['users'], data_description['timestamp']])
mask = test_data.groupby(data_description['users']).cummin()[data_description['items']] == -1
test_data_truncated = test_data[~mask]

# also get rid of users who have just 1 interaction
interaction_counts = test_data_truncated.groupby(data_description['users']).size()
users_to_keep = interaction_counts[interaction_counts >= 2].index
test_prepared = test_data_truncated[test_data_truncated[data_description['users']].isin(users_to_keep)]


In [ ]:
# split the data into validation and test by users

unique_users = test_prepared[data_description['users']].unique()
np.random.shuffle(unique_users)

# split the users into two halves
split_index = len(unique_users) // 2
users_val = unique_users[:split_index]
users_test = unique_users[split_index:]

# create val and test
test = test_prepared[test_prepared[data_description['users']].isin(users_test)]
val = test_prepared[test_prepared[data_description['users']].isin(users_val)]

To perform successive evaluation, we need to have access to the user's history in chronological order. Let's create a dictionary with users as keys

In [ ]:
test_dict = {}
for user, item, rating, timestamp in test.values:
    ...

In [ ]:
test_dict

In [ ]:
def downvote_seen_items_sequence(scores, seen_sequence):
    assert isinstance(scores, np.ndarray), 'Scores must be a dense numpy array!'
    assert scores.shape[0] == len(seen_sequence), 'Scores size is different from sequence length!'
    
    for i in range(len(seen_sequence)):
        scores[i, seen_sequence[:i + 1]] = scores.min() - 1

In [ ]:
def successive_evaluation(test_dict, model, topn=10):
    cum_hits = 0
    cum_reciprocal_ranks = 0.
    cum_discounts = 0.
    unique_recommendations = set()
    total_count = 0
    
    for user in tqdm(test_dict):
        seen_seq = ...
        test_seq = ...
        num_predictions = len(test_seq)
        if not num_predictions: # if no test items left - skip user
            continue
        scores = model.recommend_sequential(test_seq, seen_seq, user)
        downvote_seen_items_sequence(scores, test_dict[user][:-1])
        predicted_items = topn_recommendations(scores, topn=topn)
        
        hit_steps, hit_index = np.where(predicted_items == np.atleast_2d(test_seq).T)
        unique_recommendations.update(predicted_items.ravel())

        num_hits = hit_index.size
        if num_hits:
            cum_hits += num_hits
            cum_reciprocal_ranks += np.sum(1. / (hit_index+1))
            cum_discounts += np.sum(1. / np.log2(hit_index+2))

        total_count += num_predictions

    hr = cum_hits / total_count
    mrr = cum_reciprocal_ranks / total_count
    dcg = cum_discounts / total_count
    cov = len(unique_recommendations) / scores.shape[1]
    results = pd.DataFrame(
        data = {f'{model.__class__.__name__}': [hr, mrr, dcg, cov]},
        index = [f'{metric}@{topn}' for metric in ['HR', 'MRR', 'NDCG', 'COV']]
    )
    return results
        
        

# Baselines

To compare the performance of our Association and Sequential Rule methods we should also build some baselines - random and popular-based. This also serves as sanity check for our models.

In [ ]:
class Random:
    def __init__(self, model_config=None) -> None:
        # for reproducibility, not a hyperparameter
        self.seed = model_config['seed']
        self.rng = np.random.default_rng(seed=model_config['seed'])
    
    def build(self, data, data_description):
        self.n_items = data_description['n_items']
        
    def recommend(self, data, data_description):
        n_users = data.nunique()[data_description['users']]
        n_items = data_description['n_items']
        return self.rng.random((n_users, n_items))
    
    def recommend_sequential(self, target_seq, seen_seq, user):
        return self.rng.random((len(target_seq), self.n_items))

In [ ]:
class Popular:
    def __init__(self, model_config=None) -> None:
        pass
    
    def build(self, data, data_description):
        item_popularity = data[data_description['items']].value_counts()
        n_items = item_popularity.index.max() + 1
        popularity_scores = np.zeros(n_items,)
        popularity_scores[item_popularity.index] = item_popularity.values
        self.popularity_scores = popularity_scores
        
    def recommend(self, data, data_description):
        n_users = data.nunique()[data_description['users']]
        return np.tile(self.popularity_scores, (n_users, 1))
    
    def recommend_sequential(self, target_seq, seen_seq, user):
        return np.tile(self.popularity_scores, (len(target_seq), 1))

# Simple Association Rules (AR)

$$
\text{score}_{AR}(u, i) = \text{PairCount}_{AR}(i_{|I_u|}, i)
$$

$$
\text{PairCount}_{AR}(i, j) = |U_i\cap U_j|
$$

$I_u$ - interaction history of user $u$, $U_i$ - set of users who interacted with item $i$, $i_{|I_u|}$ - last item of user $u$

In [ ]:
class AR:
    def __init__(self, model_config=None) -> None:
        pass
    
    def build(self, data, data_description):
        '''
        Builds association rules matrix.
        '''
        interactions = generate_interactions_matrix(data, data_description)
        
        similarity = interactions.T.dot(interactions)
        similarity.setdiag(0)
        similarity.eliminate_zeros()
        self.rules = similarity
        
    def recommend(self, data, data_description):
        '''
        Generate scores for given data.
        '''
        # Drop duplicates, keeping the last interaction for each user
        data_sorted = data.sort_values(by=[data_description['users'], data_description['timestamp']])
        data_last_interaction = data_sorted.drop_duplicates(subset=data_description['users'], keep='last')

        interactions = generate_interactions_matrix(data_last_interaction, data_description, rebase_users=True)
        return interactions.dot(self.rules).A

    def recommend_sequential(self, target_seq, seen_seq, user):
        '''
        Generate scores for sequential evaluation - 
        subsequently add 1 item from target sequence to the seen sequence
        and generate predictions for the resulting history.
        '''
        user_profile = ...seen_seq
        test_sequence = ...target_seq
        scores = ...
        return scores

# Sequential Rules (MC)

$$
\text{score}_{SR}(u, i) = \text{PairCount}_{SR}(i_{|I_u|} \rightarrow i)
$$

$$
\text{PairCount}_{SR}(j \rightarrow i) = \sum_{v\in U} \textbf{1}[j\rightarrow_v i]
$$

where $j\rightarrow_u i$ means that item $i$ follows item $j$ in the interaction history of user $u$. $U$ is the set of users

In [ ]:
class SR:
    def __init__(self, model_config=None) -> None:
        pass
    
    def build(self, data, data_description):
        'Builds sequential rules of size two'
        rules = {}
        
        # get chronological interaction history for each user
        histories = (
            data
            .sort_values(
                by=data_description['timestamp']
                )
            .groupby(data_description['users'])[data_description['items']]
            .apply(list)
            )
        
        # count the number of pairs when item j is interacted with right after item i
        for history in histories:
            for i in range(len(history) - 1):
                if (history[i], history[i + 1]) not in rules:
                    rules[(history[i], history[i + 1])] = 0
                rules[(history[i], history[i + 1])] += 1
                
        # create a sparse matrix of sequential rules for easier recommendation
        items, values = zip(*rules.items())
        i1, i2 = zip(*items)
        matrix_shape = (data_description['n_items'], data_description['n_items'])
        similarity = coo_matrix((values, (list(i1), list(i2))), shape=matrix_shape).tocsr().T
        self.rules = similarity

    def recommend(self, data, data_description):
        '''
        Generate scores for given data.
        '''
        data_sorted = data.sort_values(by=[data_description['users'], data_description['timestamp']])
        data_last_interaction = data_sorted.drop_duplicates(subset=data_description['users'], keep='last')
        
        interactions = generate_interactions_matrix(data_last_interaction, data_description, rebase_users=True)
        return interactions.dot(self.rules).A
    
    def recommend_sequential(self, target_seq, seen_seq, user):
        '''
        Generate scores for sequential evaluation - 
        subsequently add 1 item from target sequence to the seen sequence
        and generate predictions for the resulting history.
        '''
        user_profile = ...seen_seq
        test_sequence = ...target_seq
        scores = ...
        return scores

In [ ]:
data_description_rules = {
    'n_users':training.nunique()['userid'],
    'n_items':training.nunique()['movieid'],
    'users':'userid',
    'items':'movieid',
    'feedback':'rating',
    'timestamp':'timestamp'
}

In [ ]:
ar_model = AR()
ar_model.build(training, data_description_rules)
results_ar = successive_evaluation(test_dict, ar_model)

In [ ]:
sr_model = SR()
sr_model.build(training, data_description_rules)
results_sr = successive_evaluation(test_dict, sr_model)

In [ ]:
pop_model = Popular()
pop_model.build(training, data_description_rules)
results_pop = successive_evaluation(test_dict, pop_model)

In [ ]:
rand_model = Random({'seed':0})
rand_model.build(training, data_description_rules)
results_rand = successive_evaluation(test_dict, rand_model)

In [ ]:
pd.concat([results_sr, results_ar, results_pop, results_rand], axis=1).T

What can be done to improve model's quality?
- Allow counting not consequent pairs of items (e.g. count $\textbf{A} \rightarrow \textbf{B}$ and $\textbf{A}\rightarrow \text{C} \rightarrow \text{D} \rightarrow \textbf{B}$, probably with weighting proportional to the inverse number of items between the pair)
- This also may help in the case of sparse datasets, where there is hard to mine association rules (density of ML-1m is about $4\%$, which is pretty high)

As we can see, apart from better accuracy metrics (HR and MRR), the sequential rules model provides much more diverse recommendations.

Let's look at the sparsity of rules matrices

In [ ]:
print(f'AR rules density: {ar_model.rules.size / ar_model.rules.shape[0] ** 2:.2f}')
print(f'SR rules density: {sr_model.rules.size / sr_model.rules.shape[0] ** 2:.2f}')

Here is the sparsity pattern of the left 100 $\times$ 100 corner of rules matrix

In [ ]:
fig, ax = plt.subplots(1, 2, constrained_layout=True)
ax[0].spy(ar_model.rules[:100, :100], markersize=1, label='AR')
ax[0].set_title('AR')
ax[1].spy(sr_model.rules[:100, :100], markersize=1, label='SR')
ax[1].set_title('SR')
plt.show()

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1, 2, constrained_layout=True, figsize=(15, 7))

im0 = ax[0].imshow(ar_model.rules[:100, :100].toarray(), cmap='Greys')
ax[0].set_title('AR Heatmap')
divider0 = make_axes_locatable(ax[0])
cax0 = divider0.append_axes("right", size="5%", pad=0.0)
fig.colorbar(im0, ax=ax[0], cax=cax0)

im1 = ax[1].imshow(sr_model.rules[:100, :100].toarray(), cmap='Greys')
ax[1].set_title('SR Heatmap')
divider1 = make_axes_locatable(ax[1])
cax1 = divider1.append_axes("right", size="5%", pad=0.0)
fig.colorbar(im1, ax=ax[1], cax=cax1)

plt.show()

In [ ]:
from scipy.sparse.linalg import norm

In [ ]:
plt.semilogy(sorted(norm(ar_model.rules, axis=1)), label='AR')
plt.semilogy(sorted(norm(sr_model.rules, axis=1)), label='SR')
plt.legend()
plt.show()